In [ ]:
import sys
sys.path.append(".")
import importlib
# import datasets.lip_to_word_dataset 
import util.cv_utils
# importlib.reload(datasets.lip_to_word_dataset)
importlib.reload(util.cv_utils)
# from datasets.lip_to_word_dataset import LipReadingWordDataset
from util.cv_utils import collate_fn

import torch
from torch.utils.data import Dataset
from pathlib import Path
from torch.utils.data import DataLoader

# Automatic Mixed Precision
from torch.cuda.amp import autocast, GradScaler


In [14]:
from torch.utils.data import Dataset


class LipReadingWordDataset(Dataset):
    # Implement abstract funcs
    
    # constructor
    # pass in path object of proccessed folder
    def __init__(self, processed_dir):
        self.index_map = []
        self.cache = {} # data optimization as we save loading times for redudant multiple get item calls for data, etc: epochs
        processed_dir = Path(processed_dir)
        for folder in processed_dir.iterdir():
            folder = Path(folder)
            for data_file in folder.glob("*.pth"):            
                data = torch.load(data_file, map_location= 'cpu', weights_only=False)
                num_words = len(data["y_labels"])
                for i in range(num_words):
                    self.index_map.append((data_file, i)) 
                    # load in the indicies and path of each word token, and load actual data in real time for optimization

    def __len__(self):
        # Return total amount of word tokens
        return len(self.index_map)

    def __getitem__(self, idx):
        # Return part of sequence for each word using index map (index being which word in the file)
        data_path, index = self.index_map[idx]
        if data_path not in self.cache:
            self.cache[data_path] = torch.load(data_path, map_location= 'cpu', weights_only= False)
        # load from cache
        data = self.cache[data_path]
        
        # Load data into tensors at runtime
        x_feat, x_coords, x_veloc, y_labels = data["x_feat"][index], data["x_coords"][index], data["x_veloc"][index], data["y_labels"][index]
        return torch.tensor(x_feat, dtype= torch.float32), torch.tensor(x_coords, dtype= torch.float32), torch.tensor(x_veloc, dtype= torch.float32), torch.tensor(y_labels, dtype= torch.long)
        


In [ ]:
# Load data
folder = Path(r"C:\Projects\Lip_Reading\GRID\processed")
# Daniel's Path
# folder = Path(r"C:\Users\User\OneDrive\Documents\Projects\Lip-Reading\GRID\processed")
dataset = LipReadingWordDataset(folder)

train_loader = DataLoader(
    dataset, 
    batch_size=64, #  64
    shuffle=True, 
    collate_fn=collate_fn
    )

In [17]:
# # Define a simple model
# import torch.nn as nn
# import torch.nn.functional as F

# class LipReadingModel(nn.Module):
#     def __init__(self, x_feat, x_coords, x_veloc, hidden_size, num_classes):
#         super(LipReadingModel, self).__init__()

#         input_size = x_feat + x_coords + x_veloc
#         self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        
        
#         # Double the hidden size if LSTM is bidirectional
#         self.dropout = nn.Dropout(p=0.3)  # Dropout rate of 30%
#         self.fc = nn.Linear(hidden_size * 2, num_classes)  

#     def forward(self, x_dim, x_2d1, x_2d2):
#         x = torch.cat([x_dim, x_2d1, x_2d2], dim=2) 
#         _, (h_n, _) = self.lstm(x)  
        
#         # h_n shape: (num_directions, batch, hidden_size)
#         # Concatenate both directions (forward and backward) if bidirectional
#         h_n = torch.cat([h_n[0], h_n[1]], dim=1)
#         h_n = self.dropout(h_n)
        
#         out = self.fc(h_n)      
#         return out

#     def __getitem__(self, idx):
#         x_dim = torch.tensor(self.X_dim[idx], dtype=torch.float32)
#         x2d1  = torch.tensor(self.X_2d1[idx], dtype=torch.float32)
#         x2d2  = torch.tensor(self.X_2d2[idx], dtype=torch.float32)
#         label = torch.tensor(self.y[idx], dtype=torch.long)  # labels must be long for CrossEntropyLoss
#         return x_dim, x2d1, x2d2, label

In [18]:
# CNN + RNN model
import torch.nn as nn
import torch.nn.functional as F

class LipReadingModel_CNN_BiLSTM(nn.Module):
    def __init__(self, x_feat, x_coords, x_veloc, hidden_size, num_classes):
        super().__init__()

        # Combine all feature dims
        input_size = x_feat + x_coords + x_veloc  

        # --- CNN frontend ---
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # --- BiLSTM backend --- (RNN)
        self.lstm = nn.LSTM(
            input_size=256,        # after CNN
            hidden_size=hidden_size,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
            dropout=0.3
        )

        # --- Fully connected classifier ---
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x_dim, x_2d1, x_2d2):
        # Concatenate along feature dimension
        x = torch.cat([x_dim, x_2d1, x_2d2], dim=2)  # shape: [B, T, F]

        # CNN expects (B, F, T)
        x = x.transpose(1, 2)  # -> [B, F, T]
        x = self.cnn(x)        # -> [B, 256, T]
        x = x.transpose(1, 2)  # -> [B, T, 256]

        # LSTM
        _, (h_n, _) = self.lstm(x)  
        h_n = torch.cat([h_n[-2], h_n[-1]], dim=1)  # last layer's forward & backward

        # Classifier
        out = self.fc(h_n)
        return out


In [19]:
model = LipReadingModel_CNN_BiLSTM(
    x_feat = 8,
    x_coords = 18,
    x_veloc = 18,
    hidden_size = 256, # 256
    num_classes = 55
)

In [20]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)


# TODO AJUST lr to increase loss rate of change precision

In [ ]:
# prefer gpu "cuda"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = GradScaler("cuda")
model = model.to(device)

for epoch in range(50):
    model.train()
    total_loss = 0.0
    for x1, x2, x3, y in train_loader:

        # Move data to device
        x1 = x1.to(device)
        x2 = x2.to(device)
        x3 = x3.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        with autocast("cuda"):
            outputs = model(x1, x2, x3)
            loss = criterion(outputs, y)

        # Backpropagation
        scaler.scale(loss).backward()
        
        # Unscale before clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Step with scaled
        scaler.step(optimizer)
        scaler.update()
    
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")


Epoch 1: Loss = 3.4722
Epoch 2: Loss = 1.9686
Epoch 3: Loss = 2.8434
Epoch 4: Loss = 3.8410
Epoch 5: Loss = 0.5136
Epoch 6: Loss = 4.9399
Epoch 7: Loss = 2.1898
Epoch 8: Loss = 3.0847
Epoch 9: Loss = 2.5473
Epoch 10: Loss = 0.3178
Epoch 11: Loss = 1.8199
Epoch 12: Loss = 2.3823
Epoch 13: Loss = 1.2221
Epoch 14: Loss = 0.3188
Epoch 15: Loss = 2.5369
Epoch 16: Loss = 1.9121
Epoch 17: Loss = 1.9342
Epoch 18: Loss = 0.1338
Epoch 19: Loss = 0.8934
Epoch 20: Loss = 0.0109
Epoch 21: Loss = 0.0240
Epoch 22: Loss = 0.7995
Epoch 23: Loss = 0.0599
Epoch 24: Loss = 0.1232
Epoch 25: Loss = 0.0483
Epoch 26: Loss = 0.7291
Epoch 27: Loss = 0.2674
Epoch 28: Loss = 1.3045
Epoch 29: Loss = 2.4427
Epoch 30: Loss = 0.1453
Epoch 31: Loss = 2.3729
Epoch 32: Loss = 0.8372
Epoch 33: Loss = 1.5793
Epoch 34: Loss = 0.5821
Epoch 35: Loss = 0.0854
Epoch 36: Loss = 3.0967
Epoch 37: Loss = 0.6407
Epoch 38: Loss = 1.7730
Epoch 39: Loss = 1.0298
Epoch 40: Loss = 1.0331
Epoch 41: Loss = 2.4914
Epoch 42: Loss = 0.0311
E

In [22]:
vocab = {
    "<pad>": 0,
    "<sos>": 1,
    "<eos>": 2,
    "sp": 3,
    "bin": 4,
    "lay": 5,
    "place": 6,
    "set": 7,
    "blue": 8,
    "green": 9,
    "red": 10,
    "white": 11,
    "at": 12,
    "by": 13,
    "in": 14,
    "with": 15,
    "zero": 16,
    "one": 17,
    "two": 18,
    "three": 19,
    "four": 20,
    "five": 21,
    "six": 22,
    "seven": 23,
    "eight": 24,
    "nine": 25,
    "again": 26,
    "now": 27,
    "please": 28,
    "soon": 29,
    "a": 30,
    "b": 31,
    "c": 32,
    "d": 33,
    "e": 34,
    "f": 35,
    "g": 36,
    "h": 37,
    "i": 38,
    "j": 39,
    "k": 40,
    "l": 41,
    "m": 42,
    "n": 43,
    "o": 44,
    "p": 45,
    "q": 46,
    "r": 47,
    "s": 48,
    "t": 49,
    "u": 50,
    "v": 51,
    "x": 52,
    "y": 53,
    "z": 54
}
inverse_vocab = {v: k for k, v in vocab.items()}

In [23]:
# Adding Metrics and Evaluation
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for x1, x2, x3, y in train_loader:
        x1 = x1.to(device)
        x2 = x2.to(device)
        x3 = x3.to(device)
        y = y.to(device)

        outputs = model(x1, x2, x3)
        predicted = outputs.argmax(1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

# Compute metrics
acc = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Final Training Accuracy: {acc:.2%}")

labels_list = list(range(len(vocab)))
target_names = [inverse_vocab[i] for i in labels_list]

# Optional detailed report

# print(classification_report(all_labels, all_preds, labels=labels_list, target_names=target_names))
report = classification_report(all_labels, all_preds, labels=labels_list, target_names=target_names, output_dict=True)
df = pd.DataFrame(report).transpose()  # Transpose to have labels as rows
df.to_csv("classification_report.csv", index=True)

# Optional confusion matrix
cm = confusion_matrix(all_labels, all_preds, labels = labels_list)
# print("Confusion Matrix:\n", cm)
cm_df = pd.DataFrame(
    cm,
    index=[inverse_vocab[i] for i in labels_list],      # True labels as row names
    columns=[inverse_vocab[i] for i in labels_list]     # Predicted labels as column names
)
cm_df.to_csv("confusion_matrix.csv")

# Daniel's Note:
# I've noticed that the model is not performing well, getting words mixed up 
# ex. "one" and "blue" probably due to lack of teeth and tongue data.
# Looking at the confusion_matrix.csv can help identify which words are being confused for which.



Final Training Accuracy: 91.53%


C:\Users\ryanv\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ryanv\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ryanv\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali